<a href="https://colab.research.google.com/github/orel33/bloc3/blob/master/reseaux/colab/socket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Les Sockets en Python

Ou comment programmer des applications réseaux TCP/IP, avec l'exemple du web (protocole HTTP).

## Avant Propos

* Socket : https://docs.python.org/3/library/socket.html
* HTTP : https://fr.wikipedia.org/wiki/Hypertext_Transfer_Protocol
* Tips : http://aurelien.esnard.emi.u-bordeaux.fr/teaching/doku.php?id=rx3:index
* Cours Moodle DIU : https://moodle1.u-bordeaux.fr/course/view.php?id=4713

## Comprendre le protocole du Web (HTTP)

Avec Wireshark, vérifez que vous comprenez tout de la capture ci-dessous :

=> https://github.com/diu-uf-bordeaux/bloc3/raw/master/reseaux/trace/http.pcap

Plus d'info sur le cours Moodle :  https://moodle1.u-bordeaux.fr/course/view.php?id=4713


## Prérequis

In [0]:
! apt-get -q -y install netcat

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  netcat-traditional
The following NEW packages will be installed:
  netcat netcat-traditional
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 65.1 kB of archives.
After this operation, 157 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 netcat-traditional amd64 1.10-41.1 [61.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 netcat all 1.10-41.1 [3,436 B]
Fetched 65.1 kB in 0s (506 kB/s)
Selecting previously unselected package netcat-traditional.
(Reading database ... 132702 files and directories currently installed.)
Preparing to unpack .../netcat-traditional_1.10-41.1_amd64.deb ...
Unpacking netcat-traditional (1.10-41.1) ...
Selecting previously unselected package netcat.
Preparing to unpack .../netcat_1.10-41.1_all.deb ...
Unpacking netcat (1.10-41.1)

Une requête web à la main...

In [0]:
! echo -e 'GET / HTTP/1.0\r\n\r\n' | netcat www.perdu.com 80

HTTP/1.1 200 OK
Date: Wed, 13 Nov 2019 00:43:22 GMT
Server: Apache
Upgrade: h2
Connection: Upgrade, close
Last-Modified: Tue, 02 Jul 2019 17:07:40 GMT
ETag: "39e-58cb5c83d8f00"
Accept-Ranges: bytes
Content-Length: 926
Content-Type: text/html

<!doctype html>
<html>
<head>
        <title>Site not found &middot; DreamHost</title>
        <meta http-equiv="cache-control" content="no-cache" />
        <meta name="description" content="The owner of this domain has not yet uploaded their website." />
        <link rel="stylesheet" href="//d1a6zytsvzb7ig.cloudfront.net/newpanel/css/singlepage.css" />
</head>
<body>
        <div class="page page-missing">
                <h1>Site Not Found</h1>

                <p>Well, this is awkward. The site you're looking for is not here.</p>
                <p><small>Is this your site? <a href="https://help.dreamhost.com/hc/en-us/articles/215613517" rel="nofollow">Get more info</a> or <a href="https://panel.dreamhost.com/index.cgi?tree=support.msg" rel="

## Le Module socket en Python3

=> https://docs.python.org/3.7/library/socket

In [0]:
import socket

## String versus Byte-Array 

Les fonctions send()/recv() du module *socket* ne manipulent pas des string *classiques*, mais des byte-array !

In [0]:
s0 = "coucou"           # string object       
b0 = b"coucou"          # byte array (notez le prefixe b)
b1 = s0.encode()        # convert string to byte array
s1 = b0.decode()        # convert byte array to string (utf-8 encoding)
print("s0 =", s0)
print("b0 =", b0)

s0 = coucou
b0 = b'coucou'


## Client HTTP 1.0

Pas facile de trouver encore des sites webs qui font du HTTP 1.0... Notez le message d'erreur !

In [0]:
HOST = b'www.perdu.com'
PORT = 80
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
request = b'GET / HTTP/1.0\r\n\r\n'
s.sendall(request)
answer = s.recv(1024)
print(answer.decode())
s.close()

HTTP/1.1 200 OK
Date: Tue, 12 Nov 2019 23:32:01 GMT
Server: Apache
Upgrade: h2
Connection: Upgrade, close
Last-Modified: Tue, 02 Jul 2019 17:07:40 GMT
ETag: "39e-58cb5c83d8f00"
Accept-Ranges: bytes
Content-Length: 926
Content-Type: text/html

<!doctype html>
<html>
<head>
        <title>Site not found &middot; DreamHost</title>
        <meta http-equiv="cache-control" content="no-cache" />
        <meta name="description" content="The owner of this domain has not yet uploaded their website." />
        <link rel="stylesheet" href="//d1a6zytsvzb7ig.cloudfront.net/newpanel/css/singlepage.css" />
</head>
<body>
        <div class="page page-missing">
                <h1>Site Not Found</h1>

                <p>Well, this is awkward. The site you're looking for is not here.</p>
                <p><small>Is this your site? <a href="https://help.dreamhost.com/hc/en-us/articles/215613517" rel="nofollow">Get more info</a> or <a href="https://panel.dreamhost.com/index.cgi?tree=support.msg" rel="

## Client HTTP 1.1

Dans sa version HTTP 1.1, il est necessaire de préciser le champs **Host** dans l'entête de la requête HTTP. De plus, il est utile de mettre le champs **Connection** à la valeur **close** afin de forcer a fermeture de la connexion TCP/IP, à la fin de la réponse. Voici donc comment préparer une requête HTTP vers le serveur web www.perdu.com (port 80) :

In [0]:
HOST = b'www.perdu.com'
PORT = 80
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
request = b'GET / HTTP/1.1\r\n'
request += b'Host: ' + HOST + b'\r\n'
request += b'Connection: close\r\n'
request += b'\r\n'
print(request.decode())


GET / HTTP/1.1
Host: www.perdu.com
Connection: close




Envoyer la requête et recevoir les 1024 premiers caractères de la réponse...

In [0]:
s.sendall(request)
answer = s.recv(1024)
print(answer.decode())
s.close()


Si l'on souhaite recevoir une réponse de taille variable, il faut "boucler" jusqu'à recevoir une réponse vide (caractéristique de la fermeture de connexion du serveur).

In [0]:
s.sendall(request)

while True:
    answer = s.recv(1024)
    if answer == b'': break
    print(answer.decode('utf-8'), end='')

s.close()


HTTP/1.1 200 OK
Date: Tue, 12 Nov 2019 23:47:09 GMT
Server: Apache
Upgrade: h2
Connection: Upgrade, close
Last-Modified: Thu, 02 Jun 2016 06:01:08 GMT
ETag: "cc-5344555136fe9"
Accept-Ranges: bytes
Content-Length: 204
Vary: Accept-Encoding
Content-Type: text/html

<html><head><title>Vous Etes Perdu ?</title></head><body><h1>Perdu sur l'Internet ?</h1><h2>Pas de panique, on va vous aider</h2><strong><pre>    * <----- vous &ecirc;tes ici</pre></strong></body></html>


## Client & Serveur 

**todo**